# F-strings: A format system to rule them all

#### Juan Diego Godoy Robles, PyConES 2019, Alicante

[Wikipedia](https://en.wikipedia.org/wiki/String_interpolation): Process of evaluating a string literal containing one or more *placeholders*, yielding a result in which the placeholders are replaced with their corresponding values.

In [ ]:
from datetime import date, datetime, timedelta
talk = 'f-strings'
talk_date = date(2019, 10 , 5)
me = 'juan diego'
minutes_left = 20

print (
  f'Ey Pythonists folks!, today is {talk_date:%A %d %B of %Y}.\n'
  f'I\'m {me.title()}, wellcome to this awesome {talk!r} talk.\nYou\'ll be free '
  f'at {datetime.now() + timedelta(minutes=minutes_left):%H:%M} \U0001f44d'
)

This feature was proposed in [**PEP-0498**](https://www.python.org/dev/peps/pep-0498/) which is the main source for this talk.

## How was this solved before?

We have *three methods*, all of them are still valid, `F-strings` does not deprecate any previous mechanism.

### Printf style formatting

String objects have the [*interpolation*](https://docs.python.org/3/library/stdtypes.html#printf-style-string-formatting) operator, quite similar to **C** `sprintf` behaviour.

If *format* requires more than one argument, the value must be a *tuple* and it's length should match the formatting mask or a mapping object.

In [ ]:
'%s %s' % ('Old Fashioned', 'formatting')

In [ ]:
'%(style)s %(action)s' % {'style': 'Old Fashioned', 'action': 'formatting'}

One of the downsides is that the **conversion type** should be included.

In [ ]:
'%' % 'hello'

Only `ints`, `strs`, and `doubles` can be formatted.

And, when a single value is passed in a tuple ...

In [ ]:
'%s' % ('%-formatting sucks',  'much')

We can avoid the issue by ussing a *defensive tatic*.

In [ ]:
'%s' % (('%-formatting sucks',  'much'),)

Or knowing beforehand the *number of elements to format*. **Not flexible** at all.

In [ ]:
'%s %s' % ('%-formatting sucks',  'much')

### Format String Syntax

My favorite [*method*](https://docs.python.org/3/library/string.html#formatstrings) until the arriving of the `F-strings`. In fact, they *share* part of the the syntax / mechanisms, but `format` is clearly more verbose.

In [ ]:
awesome_conf = 'PyConES'

'This {awesome_conf} is really awesome.'.format(awesome_conf=awesome_conf)

Even if we try to simplify it as far as we possibly can, the variable hangs a **little disconnected from the context**. This is quite more evident in a *side-to-side* comparision with `F-strings`.

In [ ]:
awesome_conf = 'PyConES'

print('This {} is really awesome.'.format(awesome_conf))

print(f'This {awesome_conf} is really awesome.')


### Template strings

The [*template strings*](https://docs.python.org/3/library/string.html#template-strings) was created as an alternative to the interpolation operator (very error-prone), with one [*target in mind*](https://www.python.org/dev/peps/pep-0292/): a simpler syntax to format strings (substitution could be a more appropriate term).

In [ ]:
from string import Template
Template('Hello from $this').substitute(this='Template')

*format* protocol is not supported, conversions are not possible.

More of its downsides: poor performance and low flexibility.

## F-strings

### Definition

A way to **embed expressions**, evaluated at run time inside, in string literals.

In [ ]:
type(f'{type}')

### What changes implies?

**None**, all the previous methods are still avaliable.

### Which are their advantages?

1. **Clarity**: They improve *code readability*, that's a fact.

In [ ]:
ways_2_formatx2 = 6
awesome_lang = 'Python'

print(
    'In %s we have %d ways to do our formatting'
    % (awesome_lang, int(ways_2_formatx2/2))
)
print(
    'In {awesome_lang:s} we have {ways_2_format:d} ways to do our formatting'.format(
        awesome_lang=awesome_lang, ways_2_format=(int(ways_2_formatx2/2))
    )
)
print (f'In {awesome_lang} we have {int(ways_2_formatx2/2)} ways to do our formatting')

2. **Performance**: the expression is evaluated in execution time and then combined with the literal part to return the final string, nothing else is required.

In [ ]:
import dis

def foo():
    x = 42
    y = 99
    return '{} + {} = {}'.format(x, y, x + y)

dis.dis(foo)

With `LOAD_METHOD` *Python* references the function `format` which will be called later by `CALL_METHOD`. `F-strings` is free from this *overhead*.

In [ ]:
import dis

def foo():
    x = 42
    y = 99
    return f'{x} + {y} = {x + y}'

dis.dis(foo)

We can test this in practice in a *real world example*.

In [ ]:
import timeit

timeit_times = 10000000

print(timeit.timeit("""name='PyConES';year=2019;f'{name} - {year}'""", number=timeit_times))
print(timeit.timeit("""name='PyConES';year=2019;'%s - %d' % (name, year)""", number=timeit_times))
print(timeit.timeit("""name='PyConES';year=2019;'{} - {}'.format(name, year)""", number=timeit_times))

### How does it work?

Only syntax errors can be caught at **compile time**, example: an orphan curly brace (`{` o `}`).

At **execution time** the expression will be evaluated in the context where the `F-string` appears with full access to *local* and *global variables*.

In the following example both `print` statements are **completely equivalent**.

In [ ]:
def hi():
    return 'Hello'

print(f'{hi()} world!')
print(str(hi())+ ' world!')

### How do we use it?


`F-strings` are string literals that are prefixed by the letter `f` or `F`.

Otherwise, *same rules as normal strings are applied*. That is, the string must end with the same character that it started with: if it starts with a single quote it must end with a single quote, etc.

Once *tokenized*, a `F-string` is parsed into **literal strings** and **expressions**, the code should appear within **curly braces**: `{expr}`.




To *escape* a curly brace we should double it:  `{{` or `}}`.

Comment (`#`) or escape chars (`\`) are not allowed inside an expression, this last inconvenient can be bypassed by *alternate the quoting char* or using *triple quoting*


An optional **type conversion** may be specified as the last part of the expression. These are treated the same as in `str.format()`: `!s`  calls `str()`, `!r` to `repr()` and `!a` to `ascii()`.

Optional **format specifiers** maybe be included, separated from the expression (or the type conversion, if specified) by a colon.

In [ ]:
from datetime import datetime
import decimal

width = 6
precision = 4
hour = decimal.Decimal('13.29999999999999999999')

f'''Playing with {{ {" f-strings '-) ".upper()!s:-^20} }} {datetime.now():%Y}{hour:{width}.{precision}}'''

`F-strings` can be use in *raw mode*, by adding de prefix `r` or `R` . The escape char `\` will not be interpreted. No escape processing will be done.

In [ ]:
import re

re.search(fr'=\s*{20 * 2}', 'sum=  40')

## Fun with F-strings

### Objects 

In [ ]:
import datetime

class Talk:
    def __init__(self, title, conference, date):
        self.title = title
        self.conference = conference
        self.date = date

    def __str__(self):
        return f'{self.title} ({self.conference} {self.date:%Y})'

    def __repr__(self):
        return f'{self.conference}: Today is {self.date:%A %d %B} Wellcome to {self.title!r}'


my_talk = Talk('f-strings', 'PyConES', datetime.date(2019, 10, 5))
print(f'''{Talk('f-strings', 'PyConES', datetime.date(2019, 10 , 5))!r}''')
print(f'{my_talk}')

### Exceptions

In [ ]:
try:
    print(non_existent)
except Exception as err:
    print(f'an error hapenned: {err}')

### Multiline

In [ ]:
print (
    f'F-strings provide a way to embed \'{"expressions"}\' inside string literals, '
    f'using a minimal syntax. '
)

### Ternary operator

In [ ]:
foo = None

f'{foo if foo is not None else "foo"}'

### Lambda functions

In [ ]:
f'{(lambda x: x**2)(3)}'

### List comprehensions

In [ ]:
celsius = [0, 20, 40]

[f'{c} Celsius => {1.8 * c + 32:.2f} Fahrenheit' for c in celsius]

### Handy formatting

**Note**: `format` like.

In [ ]:
left = 'left'
center = 'center'
right = 'right'

f'{left:><15}{center:-^10}{right:<>15}'

In [ ]:
from math import pi

f'Pi: {pi} - {pi:.4f}'

### Bonus track: DEBUG  >= 3.8

```python
>>>foo = 30
>>> print(f'{foo=}  {cos(radians(foo))=:.3f}')
foo=30  cos(radians(foo))=0.866
```

In [ ]:
foo = 30

print(f'{foo=}  {cos(radians(foo))=:.3f}')

## Pitfalls

### Modern Python >= 3.6

IMO This should be considered an advantage rather than a problem, let's avoid the "*Legacy Python*",  but **bear it in mind**.

### Docstrings

**Runtime evaluation** prevents `F-strings` from being used for code documentation.

### Quoting 

`F-strings` quoting syntax could be *kind of tricky*. Actually the [PEP536](https://www.python.org/dev/peps/pep-0536/) proposes a formal grammar lifting restrictions. In a (*near?*) future this expression will be valid:
`f'Magic wand: {bag['wand']:^10}'`.

This `PEP` is in *Deferred* state.

### Dicts

`format` could be more handy to use when handling `dicts`.

In [ ]:
nerd = {'name': 'Juan Diego', 'from': 'Almería'}

print('This nerd is {name} from {from}'.format(**nerd))

print(f'This nerd is {nerd["name"]} from {nerd["from"]}')

### Logging

`F-strings` **calls automatically `__str__`** object method, you can run into a performance problem because of this.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('fail')

class Dummy:
    def __init__(self, name):
        self._name = name
    def __str__(self):
        print('logging should be >= INFO')
        return self._name
    
c = Dummy('fstring')

logger.debug(f'Created: {c}')

In this case traditional interpolation may result *more convenient*.

In [ ]:
logger.debug('Created: %s', c)

### Last but not least

> There should be one-- and preferably only one --obvious way to do it.

Is it worth it? ...

In [ ]:
from datetime import date, datetime, timedelta
talk = 'f-strings'
talk_date = date(2019, 10 , 5)
me = 'juan diego'
minutes_left = 0

print (
  f'Ey Pythonists folks!, today is {talk_date:%A %d %B of %Y}.\n'
  f'I\'m {me.title()}, wellcome to this awesome {talk!r} talk.\nYou\'ll be free '
  f'at {datetime.now() + timedelta(minutes=minutes_left):%H:%M} \U0001f44d'
)

# Thanks !!
### Contact: https://klashxx.github.io/about